<a href="https://colab.research.google.com/github/a2r4vind/Machine-Learning-Models/blob/main/Implementing_NeuralNetwork_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf